In [0]:
df = spark.readStream.format('cloudFiles') \
    .option("cloudFiles.format", "csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ";")\
    .option("cloudFiles.schemaLocation", "abfss://data@databricksmths.dfs.core.windows.net/retail/checkpoint") \
    .load("abfss://data@databricksmths.dfs.core.windows.net/retail/raw")



In [0]:

(df.writeStream.format("delta")
   # checkpoint folder (important for exactly-once processing)
   .option("checkpointLocation", "abfss://data@databricksmths.dfs.core.windows.net/retail/checkpoints/bronze")
   
   # handle schema changes automatically
   .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
   .option("mergeSchema", "true")
   
   # log bad records separately
   .option("badRecordsPath", "abfss://data@databricksmths.dfs.core.windows.net/retail/badrecords")
   
   # append mode = keep adding new files
   .outputMode("append")
   
   # write as Delta table in metastore
   .table("retail_bronze")
)


In [0]:
# Read a small sample from the Bronze table
bronze_df = spark.read.table("retail_bronze")

print("📌 Bronze Table Schema:")
bronze_df.printSchema()

print("📌 Sample Bronze Records:")
display(bronze_df.limit(10))
